In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "/home/lyz/hf-models/Qwen/Qwen1.5-1.8B-Chat/",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/home/lyz/hf-models/Qwen/Qwen1.5-0.5B-Chat/")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
train = pd.read_csv('train.csv', sep='\t')
train

,table,question,answer
0,30d4b610-6eb7-4dca-a6b5-b42dc091aa4a.table,请问在所有参赛者中，谁是在合并部落（Merged Tribe）阶段被投票淘汰的第一位评委成员？,Viktor Gusev 47.the sport commentator
1,1daed2ed-ff65-44b6-897b-ef76e616d05f.table,请问在2011年爱迪生奖（Edison Awards）中，哪部影片获得了最佳浪漫电影奖项？,在2011年爱迪生奖（Edison Awards）中，最佳浪漫电影奖项没有具体列出获奖影片名称。
2,12916f1d-2fba-4b01-b6fe-acb4aebad7f0.table,请问在所提供的国家列表中，哪一个国家的国内生产总值（GDP）总量是最低的？,Comoros
3,f81876ca-d4ad-46a3-a322-a0e1b47d4991.table,在2001年的电视剧《齐天大圣孙悟空》中，陈冠希饰演的角色叫什么？,Purple Orchid
4,0b5e1eb2-4225-49a2-85f7-e82e62dd029a.table,请问在上述电影列表中，排名第一的电影是哪一部？,排名第一的电影是《Star Wars*》。
...,...,...,...
495,5cefffc1-35da-4f54-8a03-e3ee0f93d896.table,请问在1973年的比赛中，男子组的冠军是谁？,Sweden Bengt Gustafsson
496,49fb8179-40e9-4e05-826e-1e32c51495f8.table,请问在1963年，球队在联赛中的最终排名是多少？,4th
497,422da020-5866-4074-bdf9-2ec358e45b04.table,请根据以下表格数据，具体提问关于参赛选手Andrea Rothfuss在两次跑步中的排名变化...,参赛选手Andrea Rothfuss在两次跑步中都保持了第一名的排名，没有发生变化。
498,dd6e75ca-2742-4834-a595-a96374ea8b28.table,请问在所列出的各支球队中，哪支球队的进球数与失球数差距最大？,Montreal Canadiens


In [3]:
test = pd.read_csv('test.csv', sep='\t')
test

,table,question
0,da418c0c-33cd-4424-9c08-026b5e683fe7.table,在上述表格中，第25至32场比赛中，哪位选手代表欧洲队在单打比赛中战胜了美国队的选手？
1,a601205d-b66c-426c-bc31-18df6ccc6e6f.table,"在2001年发行的歌曲 ""Gainesville Rock City"" 中，它在英国音乐排行..."
2,d0c9dc11-937e-4de1-8939-e5b4e79561a2.table,"""《Arunthathi》这部电影的导演是谁？"""
3,58716f17-d58f-4211-9589-397a0ff73fc5.table,请问《Rainism》这首歌的音频制作导演是谁？
4,53a5d0ba-4b37-45e9-b2d7-565774ad4e0e.table,请问在8月28日与Burnley的比赛中，比赛的最终比分是多少？
...,...,...
1545,67924d62-6133-4e5c-b790-2fc627e4e93a.table,请根据上表提供的数据，具体描述2007年至2013年间，评分等级为“4”的学生比例在各个年份...
1546,1bd58446-4b88-492f-b91d-a733a9907851.table,请根据表格提供的信息，回答以下问题：第123号球员Paul Ames的NHL队伍是哪一支？
1547,9047b42a-8ad7-4556-b181-3c8ea16c502f.table,“表格中所列的各个车型中，哪款车型的起始价格最低？”
1548,ff078d5d-ee99-4156-aa7f-ddcbd159d83d.table,请问在这个表格所列出的音乐作品中，哪一位音乐制作人参与了最多歌曲的制作？


In [ ]:
test_answer = []
for row in test.iterrows():
    content  = ''.join(open('tables/' + row[1].table).readlines()[:50])
    prompt_template = f'请结合给定的资料客观回答问题，如果问题无法从表中回答则回复无法回答，回答结果要精简，简短直接回答结果，不要分析如何回答的，直接回答，出了答案之外不需要有其他内容。\n -资料\n {content} -提问 \n {row[1].question}'

    messages = [
        {"role": "user", "content": prompt_template[:4096]}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=50
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(row[1].question, response)
    print('')
    test_answer.append(response.strip().replace('\n', ''))

在上述表格中，第25至32场比赛中，哪位选手代表欧洲队在单打比赛中战胜了美国队的选手？ Mika Koivuniemi。

在2001年发行的歌曲 "Gainesville Rock City" 中，它在英国音乐排行榜上取得了第几位？  在2001年发行的歌曲 "Gainesville Rock City" 中，它在英国音乐排行榜上取得了第78位。

"《Arunthathi》这部电影的导演是谁？" 电影《阿伦特·坦达》和《德哈斯宾或贾奥蒂马尔》分别于1943年7月2日和1943年3月3日在现代剧院上映。电影《阿

请问《Rainism》这首歌的音频制作导演是谁？ KAT-TUN 是日本歌手组合，他们在 2004 年发行了专辑 Rainbow Mini Album，并在同年获得了日本唱片销量冠军。他们的音乐作品包括彩虹迷你专辑中的歌曲 Summary of Johnny's World 和 Kaiz

请问在8月28日与Burnley的比赛中，比赛的最终比分是多少？ 3 - 4

请问“在当前的官员中，‘主计大臣’（Lord High Treasurer）的职务状态是怎样的？” "主计大臣”(Lord High Treasurer)的职务状态为在位。

请问在1971年发行的哪一首歌曲在美国和加拿大的乡村音乐排行榜上都取得了较好的成绩？ 年份: 1967-1977
主要歌曲:
- Forbidden Fruit (单曲)
- I'd Be Your Fool Again (单曲)
- I'm In Love with My Wife (单曲)
- You T

“根据上表提供的信息，请问 'Queen Mary' 船只在1967年被出售后，它的用途发生了怎样的变化？”  "Queen Mary" 在1967年被出售后，其用途转变为一个固定型酒店船。

在篮球这项运动中，谁是发明了篮球并被机器人“Danny Defrost-Bot”所纪念的人物？ 在篮球这项运动中，美国发明家詹姆斯·纳什米修斯（James Naismith）是发明了篮球并被机器人“Danny Defrost-Bot”所纪念的人物。他在1891年发明了篮球规则

问题：安纳普尔纳I峰（Annapurna I）的首次登顶是在哪一年？ 以上为15座珠穆朗玛峰的信息。它们按照海拔高度从高到低排列，其中最高的是

In [5]:
len(test_answer)

1550

In [8]:
pd.DataFrame({'answer': test_answer}).to_csv('submit.csv', index=None)